#### Imports

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [2]:
from utils import *
from transformers import Trainer, EarlyStoppingCallback, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification

#### Main Workflow

In [3]:
dataset = load_data(path='liar', remove_all=True, concat=True)

NameError: name 'load_data' is not defined

In [ ]:
trainset = balance_dataset(dataset, split='train')
testset = dataset['test']
valset = dataset['validation']

NameError: name 'dataset' is not defined

In [ ]:
display_samples(trainset, num_samples=3)

In [ ]:
display_samples(testset, num_samples=3)

In [ ]:
display_samples(valset, num_samples=3)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')
def tokenize_dataset(data):
    return tokenizer(data['text'],
                     max_length=100,
                     truncation=True,
                     padding="max_length")

train_dataset = trainset.map(tokenize_dataset)
test_dataset = testset.map(tokenize_dataset)
val_dataset = valset.map(tokenize_dataset)

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./sentiment_transfer_learning_transformer/",
    logging_dir='./sentiment_transfer_learning_transformer/logs',
    logging_strategy='epoch',
    logging_steps=100,
    num_train_epochs=5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

In [ ]:
# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

In [ ]:
evaluate_performance(trainer, train_dataset)

In [ ]:
evaluate_performance(trainer, test_dataset)

In [ ]:
evaluate_performance(trainer, val_dataset)

In [ ]:
plot_roc_curves(model, train_dataset, label = 'Train Classifier')

In [ ]:
plot_roc_curves(model, test_dataset, label = 'Test Classifier')

In [ ]:
plot_roc_curves(model, val_dataset, label = 'Validation Classifier')